In [ ]:
import sys
import os
import hydra
from Config.config import Configuration
from AutoencoderU import AutoEncoder
import pytorch_lightning as L
from DataModule import ButterflyDataModule

notebook_dir = os.path.dirname(os.path.abspath(os.getcwd()))
config_dir = os.path.join(notebook_dir,"proyecto-transfer-learning\\Config")

sys.argv = [arg for arg in sys.argv if not arg.startswith("--")]